In [1]:
import cv2
import numpy as np
import time

In [2]:
color_ranges = {
    'white': ([0, 0, 200], [180, 20, 255]),
    'green': ([35, 50, 50], [85, 255, 255]),
    'orange': ([5, 50, 50], [15, 255, 255]),
    'yellow': ([25, 50, 50], [35, 255, 255]),
}

In [3]:
def detect_color(frame, lower, upper):
    mask = cv2.inRange(frame, np.array(lower), np.array(upper))
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return contours

In [4]:
def get_quadrant(x, y, width, height):
    quadrant_width = width // 2
    quadrant_height = height // 2

    if x < quadrant_width and y < quadrant_height:
        return 2
    elif x >= quadrant_width and y < quadrant_height:
        return 3
    elif x < quadrant_width and y >= quadrant_height:
        return 4
    elif x >= quadrant_width and y >= quadrant_height:
        return 1

In [5]:
def main(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width, frame_height))

    last_positions = {color: None for color in color_ranges}
    event_log = []

    start_time = time.time()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        current_time = time.time() - start_time

        for color_name, (lower, upper) in color_ranges.items():
            contours = detect_color(hsv_frame, np.array(lower), np.array(upper))
            for contour in contours:
                if cv2.contourArea(contour) > 100:
                    x, y, w, h = cv2.boundingRect(contour)
                    cx, cy = x + w // 2, y + h // 2
                    current_quadrant = get_quadrant(cx, cy, frame_width, frame_height)

                    if last_positions[color_name] is None:
                        last_positions[color_name] = current_quadrant
                        event_log.append((current_time, current_quadrant, color_name, 'Entry'))
                    else:
                        if last_positions[color_name] != current_quadrant:
                            event_log.append((current_time, last_positions[color_name], color_name, 'Exit'))
                            event_log.append((current_time, current_quadrant, color_name, 'Entry'))
                            last_positions[color_name] = current_quadrant

                    cv2.circle(frame, (cx, cy), 5, (0, 255, 0), -1)
                    cv2.putText(frame, f'{color_name} Q{current_quadrant}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        out.write(frame)
        cv2.imshow('Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    with open('events_log3.txt', 'w') as f:
        for event in event_log:
            f.write(f'{event[0]:.2f}, {event[1]}, {event[2]}, {event[3]}\n')

In [6]:
if __name__ == '__main__':
    video_path = 'AI Assignment video.mp4'
    main(video_path)